In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torchvision.models as models
import timm
import cupy

In [2]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print(f'Using device: {device}')

Using device: cuda


In [4]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
# Define path to your training data
train_data_path = 'AugmentedAlzheimerDataset'
# Load dataset using ImageFolder and split into train and validation
train_dataset = ImageFolder(root=train_data_path, transform=transform)

In [6]:
pip install -U ultralytics

     ---------------------------------------- 0.0/41.2 kB ? eta -:--:--
     --------- ------------------------------ 10.2/41.2 kB ? eta -:--:--
     ---------------------------- --------- 30.7/41.2 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 41.2/41.2 kB 329.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/796.9 kB ? eta -:--:--
   ------------ --------------------------- 256.0/796.9 kB 7.9 MB/s eta 0:00:01
   --------------------------------------- 796.9/796.9 kB 12.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Assuming you want to split 80% for training and 20% for validation
train_size = int(0.8 * len(train_dataset))
val_size = int(0.1*len(train_dataset))
test_size = len(train_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(train_dataset, [train_size, val_size, test_size])

In [8]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
pip install yolov5

  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
     ---------------------------------------- 0.0/88.4 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/88.4 kB ? eta -:--:--
     ------------------------------------- -- 81.9/88.4 kB 1.5 MB/s eta 0:00:01
     ---------------------------------------- 88.4/88.4 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached smmap-5.0.1-py3-none-any.whl.metadata (4.3 kB)
   ---------------------------------------- 0.0/953.4 kB ? eta -:--:--
   - -------------------------------------- 41.0/953.4 kB ? eta -:--:--
   ------- -------------------------------- 174.1/953.4 kB 2.1 MB/s eta 0:00:01
   ----------- ---------------------------- 276.5/953.4 kB 1.9 MB/s eta 0:00:01
   ---------------- -----------------

In [15]:
# Load the YOLOv5 model from Torch Hub
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Extract the backbone (CSPDarknet53) from YOLOv5
backbone = nn.Sequential(*list(yolo_model.model.children())[:-1])

# Function to calculate the output size of the backbone
def get_backbone_output_size(backbone, input_size):
    with torch.no_grad():
        dummy_input = torch.zeros(1, 3, *input_size)
        output = backbone(dummy_input)
    return output.view(1, -1).size(1)

# Calculate the size of the features coming out of the backbone
input_size = (224,224)
backbone_output_size = get_backbone_output_size(backbone, input_size)

# Define a new classification head
class YOLOv5Classification(nn.Module):
    def __init__(self, backbone, backbone_output_size, num_classes):
        super(YOLOv5Classification, self).__init__()
        self.backbone = backbone
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(backbone_output_size, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

# Instantiate the modified model
model = YOLOv5Classification(backbone, backbone_output_size, num_classes=4)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

Using cache found in C:\Users\sushant mittal/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-7-5 Python-3.10.13 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [16]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from sklearn.metrics import accuracy_score

# Define loss function and optimizer
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs=25):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_acc = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            train_acc += accuracy_score(labels.cpu(), preds.cpu())
        train_loss /= len(train_loader)
        train_acc /= len(train_loader)

        # Validation
        model.eval()
        val_loss = 0
        val_acc = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_acc += accuracy_score(labels.cpu(), preds.cpu())
        val_loss /= len(val_loader)
        val_acc /= len(val_loader)

        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, device)


Epoch 1/25, Train Loss: 3.4072, Train Acc: 0.2825, Val Loss: 1.3780, Val Acc: 0.2819
Epoch 2/25, Train Loss: 1.4084, Train Acc: 0.2823, Val Loss: 1.3769, Val Acc: 0.2819
Epoch 3/25, Train Loss: 1.3762, Train Acc: 0.2823, Val Loss: 1.3771, Val Acc: 0.2819
Epoch 4/25, Train Loss: 1.3760, Train Acc: 0.2824, Val Loss: 1.3770, Val Acc: 0.2819
Epoch 5/25, Train Loss: 1.3761, Train Acc: 0.2822, Val Loss: 1.3770, Val Acc: 0.2819
Epoch 6/25, Train Loss: 1.3760, Train Acc: 0.2823, Val Loss: 1.3770, Val Acc: 0.2819
Epoch 7/25, Train Loss: 1.3760, Train Acc: 0.2823, Val Loss: 1.3769, Val Acc: 0.2819
Epoch 8/25, Train Loss: 1.3760, Train Acc: 0.2824, Val Loss: 1.3771, Val Acc: 0.2819
Epoch 9/25, Train Loss: 1.3760, Train Acc: 0.2822, Val Loss: 1.3770, Val Acc: 0.2819
Epoch 10/25, Train Loss: 1.3760, Train Acc: 0.2823, Val Loss: 1.3769, Val Acc: 0.2819
Epoch 11/25, Train Loss: 1.3760, Train Acc: 0.2823, Val Loss: 1.3768, Val Acc: 0.2819
Epoch 12/25, Train Loss: 1.3760, Train Acc: 0.2824, Val Loss: 1

In [17]:
# Testing function
def test_model(model, test_loader, device):
    model.eval()
    test_acc = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            test_acc += accuracy_score(labels.cpu(), preds.cpu())
    test_acc /= len(test_loader)
    print(f'Test Accuracy: {test_acc:.4f}')

# Evaluate the model
test_model(model, test_loader, device)

Test Accuracy: 0.2847


In [18]:
# Save the entire model
torch.save(model, 'yolov5_classification_model.pth')

In [19]:
# Save only the model's state dictionary
torch.save(model.state_dict(), 'yolov5_classification_model_state_dict.pth')

In [20]:
# Load the entire model
model = torch.load('yolov5_classification_model.pth')
model = model.to(device)

In [21]:
# Testing function
def test_model(model, test_loader, device):
    model.eval()
    test_acc = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            test_acc += accuracy_score(labels.cpu(), preds.cpu())
    test_acc /= len(test_loader)
    print(f'Test Accuracy: {test_acc:.4f}')

# Evaluate the model
test_model(model, test_loader, device)

Test Accuracy: 0.2847
